In [ ]:
from __future__ import annotations
from scripts_for_plotting import *

In [ ]:
# CHANGE NEXT TWO LINES
configuration_file_location: str = "../input_files/tsfitpy_input_configuration.txt" # CHANGE
output_folder_location: str = "../output_files/OUTPUTCHANGEHERE/" # CHANGE
# loads all data from config file and output
config_dict = load_output_data(configuration_file_location, output_folder_location)

In [ ]:
output_results_pd_df = config_dict["output_file_df"]   # Pandas dataframe for your own use
print("Column names are:")
print(output_results_pd_df.columns.values)             # Column names if you want to plot them
output_results_pd_df

In [ ]:
# can plot your own results using column names from the data frame
# can also take any extra arguments just like plt.scatter, example with color='black' is shown below
# can also add e.g. xlim=(-5, 5) or ylim=(-1, 1) for x and y limits
# CHANGE COLUMN NAMES IF NEEDED
plot_scatter_df_results(output_results_pd_df, "Fe_H", "chi_squared", color='black')
# works in similar way, but plots the density map with the desired cmap
# s=10 stands for point size
plot_density_df_results(output_results_pd_df, "Fe_H", "chi_squared", s=10, cmap="plasma_r", alpha=0.5)
# also histogram if needed as well
plot_histogram_df_results(output_results_pd_df, "Fe_H", color='grey', bins=20, alpha=0.5, histtype='bar', ec='black')

In [ ]:
# CHANGE NEXT LINE
star_name_to_plot: str = "CHANGE_NAME_OF_STAR_TO_PLOT_HERE" # CHANGE
# plots all fitted lines for the requested star
plot_one_star(config_dict, star_name_to_plot)

In [ ]:
# can also plot all stars and all lines as well
for specname in set(output_results_pd_df["specname"].values):
    print(specname)
    plot_one_star(config_dict, specname)